# Movielens 영화 추천 실습
## 목표
MF 모델 학습 방법을 토대로, 사용자가 좋아할 만한 영화 추천 시스템을 제작해 본다. 

## 추천시스템이란?
추천시스템은 '아이템이 많고 유저의 취향이 다양할 때 유저가 소비할만한 아이템을 예측하는 모델'이다. 빅데이터를 사용하여 추천시스템을 만들 수 있으며, 추천 방식에는 _협업 필터링(collaborative filtering)_ 과 _콘텐츠 기반 필터링(contents-based filtering)_ 이 있다. 여기서는 사용자간 유사성 및 아이템 간 유사성을 기반으로 한 협업 필터링을 사용할 것이다.

## 데이터 설명
활용할 데이터셋은 추천시스템의 MNIST와 같은 __Movielens__ 데이터이다. Movielens 데이터에 대한 설명과 사용하는 방법은 다음과 같다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기별로 있다. MovieLens 1M Dataset 사용을 권장한다.        
- 별점 데이터는 대표적인 explicit 데이터이지만 implicit 데이터로 간주하고 테스트해볼 수 있다.   
- 별점을 시청횟수로 해석해서 생각한다.    
- 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외한다. 


## 순서
1. 데이터 준비와 전처리
2. 데이터 분석
3. 모델 검증을 위한 사용자 초기 정보 세팅
4. 모델에 활용하기 위한 전처리
5. CSR matrix 만들기
6. 모델 설계 및 학습
7. 사용자가 좋아하는 영화와 비슷한 영화 추천하기
8. 사용자가 좋아할만한 영화 추천하기

## 1. 데이터 준비와 전처리

### 데이터 다운로드
MovieLens 1M Dataset 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축 해제한다.

```python
1) wget으로 데이터 다운로드
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 옮김
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

3) 작업디렉토리로 이동
$ cd ~/aiffel/recommendata_iu/data

4) 압축 해제
$ unzip ml-1m.zip
```
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 정리되어 있어 전처리가 노드보다 쉬웠다.

### 필요한 라이브러리와 데이터 불러오기

In [1]:
# 필요한 라이브러리 임포트
import pandas as pd
import os

# Movielens 데이터 불러오기
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']  # 임의로 지정한 컬럼명

# Movielens 데이터 읽기, 컬럼 이름은 'ratings_cols'
# '::'로 분리
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 데이터 전처리
유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하므로 3점 이상만 남긴다.

In [2]:
# 3점 이상만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


별점을 시청횟수로 해석하므로 rating 컬럼의 이름을 count로 바꾼다.

In [3]:
# rating 컬럼의 이름을 count로 바꾼다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
# 확인
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

영화 제목을 보기 위해 메타 데이터를 읽어온다.

In [5]:
# 메타 데이터 읽기
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


> 참고: [메타데이터](https://ko.wikipedia.org/wiki/%EB%A9%94%ED%83%80%EB%8D%B0%EC%9D%B4%ED%84%B0#:~:text=%EB%A9%94%ED%83%80%EB%8D%B0%EC%9D%B4%ED%84%B0(metadata)%EB%8A%94%20%EB%8D%B0%EC%9D%B4%ED%84%B0,purpose%22%EB%9D%BC%EA%B3%A0%EB%8F%84%20%EC%A0%95%EC%9D%98%ED%95%9C%EB%8B%A4.)           
데이터에 관한 구조화된 데이터로, 다른 데이터를 설명해 주는 데이터이다. 대량의 정보 가운데에서 찾고 있는 정보를 효율적으로 찾아내서 이용하기 위해 일정한 규칙에 따라 콘텐츠에 대하여 부여되는 데이터이다. 어떤 데이터 즉 구조화된 정보를 분석, 분류하고 부가적 정보를 추가하기 위해 그 데이터 뒤에 함께 따라가는 정보를 말한다.

## 2. 데이터 탐색
- 유니크한 데이터의 개수(영화, 사용자)
- 가장 인기있는 영화 30개
- 기타

### 유니크한 데이터의 개수
pandas.DataFrame.nunique()을 사용하여 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아본다. 

In [6]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [7]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

파일 ratings에 있는 유니크한 영화는 3628개이고, 유니크한 사용자는 6039명이다.   

### 가장 인기 있는 영화 30개 검색
이제 가장 인기 있는 영화 30개를 검색해본다. 검색 후 숫자로 이루어진 movie_id가 나왔다. 영화 이름으로 검색하기 위해 두 개의 데이터프레임 ratings과 movies를 병합한 후, 다시 검색한다.

In [8]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [9]:
# 데이터 병합
data = pd.merge(ratings, movies, how='inner', on='movie_id')

In [10]:
# 확인
data.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


사용하는 컬럼만 남기고 나머지 컬럼은 제외한다.

In [11]:
# 필요없는 컬럼 제거
using_cols = ['user_id', 'title', 'count']
data = data[using_cols]
data.head(10)

,user_id,title,count
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5
5,18,One Flew Over the Cuckoo's Nest (1975),4
6,19,One Flew Over the Cuckoo's Nest (1975),5
7,24,One Flew Over the Cuckoo's Nest (1975),5
8,28,One Flew Over the Cuckoo's Nest (1975),3
9,33,One Flew Over the Cuckoo's Nest (1975),5


검색을 쉽게하기 위해 title을 소문자로 바꾸었다.

In [12]:
# 소문자로 변경
data['title'] = data['title'].str.lower() 

인기 있는 영화 30개를 검색하였다.

In [13]:
# 가장 인기 있는 영화 30개(인기순)
datas = data.groupby('title')['user_id'].count()
datas.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

### 기타
다른 사항에 대해서도 탐색해 보았다.

In [14]:
# 유저별 몇 개의 영화를 보는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

6039명의 사용자들은 평균 138개의 영화를 보고, 최대 1968개의 영화를 보았다.  

In [15]:
# 사용자별 count 횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['count'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

6039명의 사용자들은 영화에 평균 4.056 정도의 평점을 주었다. 그러나 위에서 3점 미만의 평점을 제거하였다는 것을 고려해야 한다. 

## 3. 모델 검증을 위한 사용자 초기 정보 세팅

추천시스템은 사용자의 영화 취향과 비슷한 영화를 추천받기 위해 사용자 취향의 영화 5개를 입력받는다. 여기서도 추천 시스템 모델의 추후 검증을 위해 선호하는 영화를 5가지 골라서 기존 data에 추가한다. 주의할 점은 영화 이름은 데이터셋에 있는대로 넣어야 한다는 것이다. 즉 ```'title (year)'```의 형태로 영화 이름을 넣어야 한다. 

In [16]:
# 선호 영화 이름을 리스트로 생성
my_favorite = ['jane eyre (1996)' , 'persuasion (1995)' ,'lion king, the (1994)' ,'little women (1994)' ,'forrest gump(1994)']

# 위에서 만든 데이터프레임의 형태로 컬럼 5개짜리 데이터프레임을 만든다.
# 'EJ'라는 user_id가 위의 영화에 평점을 5점씩 주었다고 가정
my_movielist = pd.DataFrame({'user_id': ['EJ']*5, 'title': my_favorite, 'count':[5]*5})

if not data.isin({'user_id':['EJ']})['user_id'].any():  # user_id에 'EJ'이라는 데이터가 없다면
    data = data.append(my_movielist)          # 임의로 만든 my_favorite 데이터 추가 

data.tail(10)       # 잘 추가되었는지 확인

,user_id,title,count
836473,5851,one little indian (1973),5
836474,5854,slaughterhouse (1987),4
836475,5854,"promise, the (versprechen, das) (1994)",3
836476,5938,"five wives, three secretaries and me (1998)",4
836477,5948,identification of a woman (identificazione di ...,5
0,EJ,jane eyre (1996),5
1,EJ,persuasion (1995),5
2,EJ,"lion king, the (1994)",5
3,EJ,little women (1994),5
4,EJ,forrest gump(1994),5


## 4. 모델에 활용하기 위한 전처리
위에 보이는 데이터프레임을 보면 새로 추가한 데이터는 user_id와 movie_id의 값이 기존 데이터와 다른 것을 알 수 있다. 따라서 데이터의 관리를 쉽게 하기 위해 기존 데이터와 같게 번호를 붙여야 한다(indexing). 

pandas.DataFrame.unique()를 사용하여 특정 컬럼에 포함된 유니크한 데이터만 모은 후, user_id와 title에 각각 번호를 붙여 인덱싱한다. 

In [17]:
# 고유한 유저, 영화를 찾아냄
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 영화를 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

인덱싱이 잘 되었는지 확인해 본다. 

In [18]:
# 확인
print(user_to_idx['EJ'])    # 6039명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 함
print(title_to_idx['little women (1994)'])

6039
959


아래는 인덱싱을 통해 데이터 컬럼 내의 값을 바꾸는 코드이다. dictionary.get(딕셔너리의 value를 구하는 코드)을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구한다(df.map() 사용). 

In [19]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구함 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()  # 인덱싱되지 않은 row가 있다면 인덱스가 NaN이므로 dropna()로 제거 

if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었으면,
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체 
else:
    print('user_id column indexing Fail!!')

# title 컬럼도 동일한 방식으로 인덱싱 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,count
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,2591,5
1,6039,1349,5
2,6039,330,5
3,6039,959,5


## 5. CSR matrix 만들기
데이터를 CSR Matrix에 맞게 바꾼다. csr_matrix를 만드는 방법은 다양하게 있는데, [여기](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)와 노드를 참고하여 만들었다. 
```python
csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]
```

CSR matrix 만들 때, 노드에 쓰여 있는 것처럼 data.count로 하였더니 'len() of unsized object'라는 메시지가 나왔다. 이것저것 해보다가 data['count']로 바꾸니 동작하였다. (data.count를 출력해보니 원하는 count 컬럼의 값 뿐 아니라 모든 컬럼의 값이 출력되었다.) 

In [20]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data['count'], (data.user_id, data.title)), shape=(num_user, num_movie))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 6. 모델 설계 및 학습
Matrix Factorization 모델을 implicit 패키지를 사용하여 학습시킨다. 
- implicit 패키지는 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있다. 
- Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식을 사용한다.

In [21]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있다. 학습 내용과는 무관
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

- [AlternatingLeastSquares 클래스](https://implicit.readthedocs.io/en/latest/als.html)의 \_\_init\_\_ 파라미터    
 - factors(int, optional) : 유저와 아이템의 벡터의 차원
 - regularization(float, optional) : 과적합을 방지하기 위한 정규화 값
 - use_gpu(bool, optional) : GPU를 사용 여부
 - iterations(int, optional) : epochs        
 - regularization과 iteration이 증가하면 학습데이터를 잘 학습하지만 과적합의 우려가 있다. 
 - factors를 늘리거나 iterations를 늘려보아도 좋다.
 
파라미터를 바꿔가면서 모델 학습을 시켰다. factor가 작으면 선호영화에 포함된 영화 벡터와의 내적값이 너무 낮았다. factor=500 정도했을 때, 0.5정도의 값이 나왔다. factor가 그 이상이면 내적값이 낮아졌다. regularization과 iteration를 바꾸었을 때는 내적값이 조금씩 변화되므로 factor가 학습에 큰 영향을 미치는 것 같다.

In [99]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=500, regularization=0.4, use_gpu=False, iterations=100
                                    , dtype=np.float32)

In [100]:
# als 모델은 input으로 (item X user) 꼴의 matrix를 받기 때문에 Transpose한다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3629x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [101]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/100 [00:00<?, ?it/s]

모델을 통해 'EJ'의 벡터와 'little women (1994)'의 벡터를 만든다.

In [102]:
# 모델로 만든 벡터 
EJ, little_women = user_to_idx['EJ'], title_to_idx['little women (1994)']
EJ_vector, little_women_vector = als_model.user_factors[EJ], als_model.item_factors[little_women]

초기 세팅에서 집어넣었던 선호영화 리스트에 포함된 영화 벡터와 사용자 벡터를 내적시켰다.

In [103]:
# 선호도 파악 
np.dot(EJ_vector, little_women_vector)

0.5559981

선호영화 리스트 외의 좋아하는 영화를 넣어 내적값을 구하고, 좋아하지 않는 영화의 내적값을 구해 어떤 결과는 내는지 확인해 보았다. 

In [104]:
# 좋아하는 영화에 대한 선호도 예측
emma = title_to_idx['emma (1996)']
emma_vector = als_model.item_factors[emma]
np.dot(EJ_vector, emma_vector)

0.30700466

In [105]:
# 좋아하지 않는 영화에 대한 선호도 예측
assassins = title_to_idx['assassins (1995)']
assassins_vector = als_model.item_factors[assassins]
np.dot(EJ_vector, assassins_vector)

-0.040198512

좋아하는 영화에 대한 선호도가 그리 높게 나오지 않았다. 좋아하지 않는 영화에 대한 선호도는 음수가 나왔다. 

## 7. 내가 좋아하는 영화와 비슷한 영화 추천하기

AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 좋아하는 영화와 비슷한 영화를 찾는다.  

In [106]:
# 좋아하는 영화와 비슷한 영화 찾는 함수
def get_similar_movie(movie_name: str):
    title_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(title_id)   # 결과는 (영화  id, 유사도) Tuple 로 반환
    idx_to_title = {v:k for k,v in title_to_idx.items()}  # title_to_idx 뒤집기
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]  # index로부터 영화 이름을 얻는 dict 생성
    return similar_movie

In [107]:
get_similar_movie('jane eyre (1996)')

['jane eyre (1996)',
 'forrest gump(1994)',
 'moll flanders (1996)',
 'washington square (1997)',
 'persuasion (1995)',
 'emma (1996)',
 'small wonders (1996)',
 'dreaming of joseph lees (1998)',
 'journey of august king, the (1995)',
 "'night mother (1986)"]

선호영화 리스트에 있는 영화를 하나 불러와 비슷한 영화를 찾았다. 영화가 어떤 내용인지 모르기 때문에 장르를 찾아 비교해 보았다. 

|movie name                        |genre      |sub-genre|subs-genre|
|:--------------------------------:|:---------:|:-------:|:--------:|
|Jane Eyre (1996)                  |Drama      |Romance  |          |
|forrest gump(1994)                |Comedy     |Romance  |War       |
|Moll Flanders (1996)              |Drama      |         |          |
|Washington Square (1997)          |Drama      |         |          |
|Persuasion (1995)                 |Romance    |         |          |
|Small Wonders (1996)              |Documentary|         |          |
|Dreaming of Joseph Lees (1998)    |Romance    |         |          |
|Journey of August King, The (1995)|Drama      |         |          |
|Night Mother (1986)               |Drama      |         |          |

장르가 비슷한 것으로 보아 좋아하는 영화와 비슷한 영화를 잘 찾은 것 같다. 다른 장르의 영화도 잘 찾는지 확인해 본다.

In [110]:
get_similar_movie('presidio, the (1988)')

['presidio, the (1988)',
 'for your eyes only (1981)',
 'nighthawks (1981)',
 'tequila sunrise (1988)',
 'view to a kill, a (1985)',
 'heartbreak ridge (1986)',
 'no mercy (1986)',
 'running scared (1986)',
 'death wish (1974)',
 'fatal beauty (1987)']

|movie name                        |genre      |sub-genre|subs-genre|
|:--------------------------------:|:---------:|:-------:|:--------:|
|Presidio, The (1988)|Action|  |  |
|For Your Eyes Only (1981)|Action|  |  |
|Nighthawks (1981)|Action|Drama|  |  |
|Tequila Sunrise (1988)|Action|Romance|Thriller|
|View to a Kill, A (1985)|Action|  |  |
|Heartbreak Ridge (1986)|Action|War|  |
|No Mercy (1986)|Action|Thriller|  |
|Running Scared (1986)|Action|Comedy|  |
|Death Wish (1974)|Action|Drama|   |
|Fatal Beauty (1987)|Action|Crime|  |

다음 영화도 장르가 비슷한 것으로 보아 비슷한 영화를 잘 찾는 것 같다. 마지막으로 다른 장르의 영화도 하나 골라 확인해 본다. 

In [111]:
get_similar_movie('firestarter (1984)')

['firestarter (1984)',
 'christine (1983)',
 'cujo (1983)',
 'children of the corn (1984)',
 'poltergeist ii: the other side (1986)',
 'pet sematary (1989)',
 'house ii: the second story (1987)',
 'creepshow 2 (1987)',
 'friday the 13th (1980)',
 "cat's eye (1985)"]

|movie name                        |genre      |sub-genre|
|:--------------------------------:|:---------:|:-------:|
|Firestarter (1984)|Horror|Thriller|  
|Christine (1983)|Horror|   |  
|Cujo (1983)|Horror|Thriller|  
|Children of the Corn (1984)|Horror|Thriller|  
|Poltergeist II: The Other Side (1986)|Horror|Thriller|  
|Pet Sematary (1989)|Horror|  |  
|House II: The Second Story (1987)|Comedy|Horror|  
|Creepshow 2 (1987)|Horror|  |  
|Friday the 13th (1980)|Horror|  |
|Cat's Eye (1985)|Horror|  |  

이 영화도 장르가 비슷한 것으로 보아 비슷한 영화를 잘 찾는 것 같다. 

## 8. 좋아할 만한 영화 추천하기
AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 좋아할 만한 영화를 추천받는다. filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument이다. 

In [126]:
user = user_to_idx['EJ']
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(352, 0.3070047),
 (10, 0.21866518),
 (209, 0.18122017),
 (357, 0.18108028),
 (2227, 0.17683086),
 (1350, 0.17525572),
 (264, 0.16552544),
 (588, 0.15481822),
 (934, 0.14802681),
 (33, 0.14681554),
 (1654, 0.14623633),
 (30, 0.1381732),
 (485, 0.13603848),
 (49, 0.13220641),
 (1325, 0.12945832),
 (1516, 0.12927799),
 (894, 0.11922239),
 (34, 0.117969476),
 (2178, 0.115400285),
 (631, 0.11523483)]

벡터로 받은 추천 영화를 영화 이름으로 변화시킨다. 

In [128]:
idx_to_title = {v:k for k, v in title_to_idx.items()}
[idx_to_title[i[0]] for i in movie_recommended] 

['emma (1996)',
 'beauty and the beast (1991)',
 'wings of the dove, the (1997)',
 'sense and sensibility (1995)',
 'end of the affair, the (1999)',
 'mansfield park (1999)',
 'mrs. brown (her majesty, mrs. brown) (1997)',
 'roman holiday (1953)',
 'circle of friends (1995)',
 'aladdin (1992)',
 'shadowlands (1993)',
 'antz (1998)',
 'age of innocence, the (1993)',
 'secret garden, the (1993)',
 'purple rose of cairo, the (1985)',
 'how to make an american quilt (1995)',
 'laura (1944)',
 'mulan (1998)',
 'twelfth night (1996)',
 'mr. mom (1983)']

AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하여 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인한다.

In [131]:
emma = title_to_idx['emma (1996)']
explain = als_model.explain(user, csr_data, itemid=emma)

추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수)를 반환한다. 

In [132]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('persuasion (1995)', 0.1250065169409618),
 ('little women (1994)', 0.10246030459591465),
 ('jane eyre (1996)', 0.09699384802665109),
 ('forrest gump(1994)', 0.003821273391864324),
 ('lion king, the (1994)', -0.021920398911673383)]

|movie name                        |genre      |sub-genre|subs-genre|
|:--------------------------------:|:---------:|:-------:|:--------:|
|Jane Eyre (1996)|Drama|Romance|   |
|Persuasion (1995)|Romance|  |  |
|lion king, the (1994)'|Animation|Children's|Musical
|little women (1994)'|Drama|  |  |
|forrest gump(1994)|Comedy|Romance|War|

emma의 장르는 __Comedy|Drama|Romance__ 이다. 위의 표와 비교해보았을 때, 영화의 subs-genre-> sub-genre-> genre를 비교하여 기여도를 따지는 것 같다. Lion King의 경우 같은 장르가 하나도 없으므로 음수가 나온 것 같다.

In [133]:
aladdin = title_to_idx['aladdin (1992)']
explain = als_model.explain(user, csr_data, itemid=aladdin)

In [134]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('lion king, the (1994)', 0.15884641997578575),
 ('jane eyre (1996)', -0.000265039091614462),
 ('little women (1994)', -0.0016224608176249045),
 ('forrest gump(1994)', -0.0017425543071893216),
 ('persuasion (1995)', -0.008781723465079067)]

aladdin의 장르는 __Animation|Children's|Comedy|Musical__ 이다. lion king과 같은 장르를 가지고 있어서 추천된 것 같다. 다른 영화들은 겹치는 장르가 없으므로 음수가 나온다.

In [137]:
quilt = title_to_idx['laura (1944)']
explain = als_model.explain(user, csr_data, itemid=quilt)

In [138]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('little women (1994)', 0.04025746243456967),
 ('persuasion (1995)', 0.038347182496363985),
 ('lion king, the (1994)', 0.025323728667959473),
 ('jane eyre (1996)', 0.013579279739006793),
 ('forrest gump(1994)', 0.001382889044907614)]

특이하게도 laura의 장르는 __Crime|Film-Noir|Mystery__ 이다. 이 영화는 선호영화의 장르와는 겹치는 것이 없는데도 추천되었다. 그래서인지 기여도가 모두 낮은 것을 발견할 수 있다. 

추천된 다른 영화들의 장르를 확인했을 때, laura를 제외하고 선호영화의 장르와 겹치는 것을 발견했다. 

## 마무리
MF 모델을 사용하여 추천시스템을 만들어 보았다. 모델 학습의 결과가 아주 좋지 않아서 제대로 추천할까 걱정했는데, Laura 외에는 비슷한 장르로 추천이 되었다. Laura의 경우는 선호 영화의 기여도가 냊은데도 왜 추천이 되었을까 하는 궁금증이 든다. (실제로 범죄/추리물을 좋아하지 않는다.)     
이 프로젝트에서 아쉬운 점이 있다면 데이터가 오래된 것이고, 영화에 대해 잘 몰라 추천된 영화가 정말 내가 좋아할만한 영화인지 판단할 수 없다는 것이다. 그래도 좋아하는 장르인 시대물이나 애니메이션이 많이 포함된 것으로 보아 추천이 잘 된 것 같기는 하다. 


## 루브릭
|평가문항|	상세기준|결과|
|:------:|:--------:|:---:|
|CSR matrix가 정상적으로 만들어졌다.|사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|Y     |
|MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|Y     |
|비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.| Y    |

## 후기
### 이번 프로젝트에서 어려웠던 점
이번 노드는 쉽게 설명이 써 있고 추가적으로 해야할 것이 없기 때문에 그렇게 어려운 부분은 없었다. 그러나 파라미터를 변경하여도 모델의 성능이 크게 변하지 않아서 적당한 파라미터를 찾는 것이 어려웠다.

### 프로젝트를 진행하면서 알게된 점
- 필터링의 종류, MF 모델, CSR 행렬 등 유용한 개념에 대해 알게 되었다.
- 인덱싱, 인덱싱을 뒤집기 등 유용한 코드를 배운 것 같다. 

### 프로젝트를 진행하면서 아직 모호한 점
- 모델의 성능 지표에 대해 노드에서 언급하였다. 그 부분에 대해서 찾아보았는데, 실제 사용자가 추천시스템에 대해 어떻게 반응하는지를 알아야 추천시스템의 성능을 판단할 수 있을 것 같다. 

### 자기 다짐
노드에서는 이번에 사용한 MF 모델 외에도 다양한 모델이 있다고 하였다. 기회가 된다면 오늘 배운 모델 외의 다양한 모델에 대해서도 공부해보고 싶다. 